In [2]:
import pandas as pd
import os
import numpy as np

In [8]:
PATH = "./wandb_autoencoder_0506.csv"
dataset = pd.read_csv(PATH)

# del dataset["Created"], dataset["Runtime"], dataset["GradC"], dataset["act_B"], dataset["act_F"], dataset["epochs"], dataset["loss_scale_ssl"]
# del dataset["User"], dataset["Group"], dataset["Job Type"], dataset["Tags"], dataset["Sweep"], dataset["Description"], dataset["GPU Count"], dataset["best_epoch"], dataset["num_chn"], dataset["tmax"], dataset["warmup"], dataset["eta_min"]
# if one of the above colum is in the dataset, delete it
for col_name in dataset.columns:
    if col_name in ["Created", "Runtime", "GradC", "act_B", "act_F", "epochs", "loss_scale_ssl", "User", "Group", "Job Type", "Tags", "Sweep", "Description", "GPU Count", "best_epoch", "num_chn", "tmax", "warmup", "eta_min"]:
        del dataset[col_name]

print("Size of dataset: ", len(dataset))
dataset = dataset[dataset["State"]=="finished"]
# dataset = dataset[dataset["architecture"]=="cnn_pool"]
# dataset = dataset[dataset["batchsize"]==64]
# dataset = dataset[dataset["fw_bn"]==0]
print("Size of dataset: ", len(dataset))

Size of dataset:  406
Size of dataset:  397


In [9]:
dataset[dataset.method=="CCL"].dataset

130    STL10
131    STL10
132    STL10
133    STL10
134    STL10
       ...  
392    STL10
394    STL10
395    STL10
396    STL10
398    STL10
Name: dataset, Length: 216, dtype: object

In [13]:
for task in ["STL10"]:
    for method in ["BP", "CCL"]:
        ds = dataset[dataset["dataset"] == task]
        ds = ds[ds["method"] == method]
        ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
        ds_count = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).count()
        ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
        ds_mean = ds_mean[ds_count["test_loss"] > 2]
        ds_std = ds_std[ds_count["test_loss"] > 2]
        try:
            row_with_highest_val_acc = ds_mean["test_loss"].idxmin()
        except:
            continue
        print("dataset: ", task, " method: ", method)
        print(row_with_highest_val_acc)
        print(f"mean: {ds_mean.loc[row_with_highest_val_acc]['test_loss']*100:.4f} std: {ds_std.loc[row_with_highest_val_acc]['test_loss']*100:.4f} count: {ds_count.loc[row_with_highest_val_acc]['test_loss']}")
        # if task == "CIFAR10" and method == "CCL":
        #     assert 0 == 1
        # if task == "STL10_cls" and method == "CCL":
        #     assert 0 == 1

dataset:  STL10  method:  BP
(0.3, 0.01, 0.01, 0.5, 0.0, 1e-05, 0.0, 0.0, 2, 1)
mean: 0.5857 std: 0.0116 count: 15
dataset:  STL10  method:  CCL
(0.1, 0.01, 0.01, 0.9, 0.9, 0.0001, 0.0001, 0.0001, 2, 1)
mean: 0.8611 std: 0.0032 count: 18


/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_48737/2443707021.py:5: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_mean = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).mean()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_48737/2443707021.py:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.std is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_std = ds.groupby(["grad_clip_F", "lr_F", "lr_B", "mmt_F", "mmt_B", "wd_F", "wd_B", "wd_B", "bw_bn", "loss_scale_C"]).std()
/var/folders/q7/s2kfc0s17x901pb1hbky39mh0000gn/T/ipykernel_48737/2443707021.py:5: FutureWarning: The defa